Understanding Chat-Scenes Text generation and comparing it to PointLLM

In [ ]:
#* 1.1 We have the generation function  in Chat-Scene

outputs = self.llama_model.generate(
    inputs_embeds=wrapped_embed,
    max_new_tokens=self.max_txt_len,
    # stopping_criteria=stopping_criteria,
    num_beams=5,
    # do_sample=True,
    min_length=1,
    # top_p=0.9,
    repetition_penalty=3.0,
    length_penalty=1,
    temperature=1.0,
    customized_mask=attention_mask
)
output_token = outputs[0]
output_text = self.llama_tokenizer.decode(output_token)
output_text = output_text.split(self.end_sym)[0]
output_text = output_text.replace('  ', ' ').replace(' .', '.').strip()
output_text = recover_caption(output_text, assigned_ids[i].tolist())
output_texts.append(output_text)
print("output", output_text)


In [ ]:
#* 1.2 And similarly in PointLLM
output_ids = model.generate(
    input_ids,
    point_clouds=point_clouds,
    do_sample=True,
    temperature=1.0,
    top_k=50,
    max_length=2048,
    top_p=0.95,
    stopping_criteria=[stopping_criteria])


Each component in Chat-Scenes generation:

1. input_embeds = wrapped_embed = torch.cat([p_0_embed, object_list_embed, p_1_embed, prompt_embed], dim=1):

In [ ]:

#p_0 and p_1 embedding:
#Embeddings giving the LLM model an idea about the situation and its role
p_0_embed = self.system + " " + self.instruction 

self.system: A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
self. The conversation centers around an indoor scene. Object information:

p_1_embed = self.role[0]  #  => ("USER", "ASSISTANT")

# prompt_emb:
# Replace template with the predicted object IDS, then tokenize the text_embeddings for the LLM
tmp_prompt = f" {custom_prompt[i]} {self.role[1]}: "
tmp_prompt = update_caption(tmp_prompt, assigned_ids[i])  # replace placeholders
prompt_embed = self.get_text_emb(tmp_prompt, device=device) # embedd into LDM space

# object_list_embed 
# Generate embeddings of the objects by comining object, image and scence embeddings, and object ids and asigned ids.


object_list_embed = self.get_object_list_embed(
    proj_object_embed[i],                                       # linear layer on x, _ =  encode_object_feat(scene_feat, scene_img_feat, scene_locs)
    proj_object_img_embed[i] if self.add_img_token else None,  # linear layer on of _, y =  encode_object_feat()
    proj_scene_embed[i] if self.add_scene_token else None,      # also combination of multiple projection of encode_object_feat()
    scene_mask[i],    # Is given
    obj_ids[i],       # Is given
    assigned_ids[i]   # Is given
)

    # object_list proj_scene_emb:
    # A series of transformation. We need to pass object_emb
    obj_embed = self.scene_init_proj(object_embed)
    mins, maxs = self.get_min_max_coord(scene_locs[:, :, :3], scene_mask)
    pos_embed = self.pos_embedding(scene_locs[:, :, :3], input_range=[mins, maxs])
    pos_embed = self.pos_proj(pos_embed)
    scene_embed = obj_embed + pos_embed
    scene_embed = self.relation_module(scene_embed, src_key_padding_mask=~scene_mask)
    proj_scene_embed = self.scene_proj(scene_embed)

    # proj_object_embed & roj_object_img_embed
    # two easy linear layers
    object_embed, object_img_embed = self.encode_object_feat(scene_feat, scene_img_feat, scene_locs)
    device = object_embed.device
    batch_size, obj_num = object_embed.shape[:2]
    proj_object_embed = self.object_proj(object_embed)
    proj_object_img_embed = self.object_img_proj(object_img_embed)

Problem, scence_feat, scene_img_scene_locs and thus object embeddings are all pre calculated with no function in the project to calculate them on the fly f.e:
{'objects': ['wall', 'wall', 'door', 'table', 'board', 'board', 'window', 'backpack', 'trash can', 'chair', 'window', 'object', 'whiteboard', 'window', 'whiteboard', 'floor', 'shelf'], 'locs': tensor([[ 0.4843,  3.0964,  1.1317,  2.9031,  0.4073,  1.5365],
        [ 1.9221,  0.2821,  0.9044,  0.3380,  5.9967,  1.7973],
        [ 1.9491,  2.5492,  1.0719,  0.1746,  1.1071,  1.3984],
        [ 1.0610, -0.3603,  0.4325,  1.6986,  0.8621,  0.8731],
        [ 1.8779,  1.4446,  0.6964,  0.1810,  1.3372,  1.0040],
        [ 0.1698,  2.9925,  1.0072,  0.7949,  0.4539,  1.3176],
        [ 1.9896,  0.4923,  0.8999,  0.3169,  0.8710,  1.0059],
        [ 1.1553, -0.2838,  0.9101,  0.5141,  0.4913,  0.2214],
        [ 0.2111, -0.3626,  0.2123,  0.2265,  0.3294,  0.4584],
        [ 1.2761, -0.5555,  0.2638,  0.6453,  0.4383,  0.4702],
        [ 1.9756, -1.1988,  0.8657,  0.2644,  0.6797,  1.1207],
        [ 1.8489, -1.3361,  0.0752,  0.1166,  0.1187,  0.0509],
        [ 1.3893, -1.7899,  0.6864,  0.8835,  0.3986,  1.3962],
        [ 2.0321, -2.6150,  0.3549,  0.2220,  0.4936,  0.1530],
        [ 1.2083, -2.6693,  0.5254,  1.7060,  0.3711,  1.1552],
        [ 0.5395, -0.4110,  0.2262,  2.9660,  7.4137,  0.6083],
        [-1.0287, -0.3186,  0.9265,  0.6522,  6.6778,  1.9152]])}